In [1]:
import os
import glob
import uuid
import cv2
import numpy as np
import av
from tqdm import tqdm

def get_frames_with_pyav(
    video_root,
    output_dir,
    frame_interval=2,
    difference_threshold=0.005,
    std_threshold=5,
    range_threshold=10,
    filename_mode="file"
):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    video_path_list = glob.glob(os.path.join(video_root, "**", '*.mp4'), recursive=True)
    
    

    for idx, video_path in enumerate(tqdm(video_path_list, desc="Processing videos", unit="video")):
        try:
            tqdm.write(f"\n正在处理视频文件 ({idx + 1}/{len(video_path_list)}): {video_path}")
            container = av.open(video_path)

            stream = container.streams.video[0]
            fps = float(stream.average_rate)
            if fps == 0:
                tqdm.write(f"无法获取视频 {video_path} 的帧率，已跳过。")
                continue

            frame_interval_frames = int(frame_interval * fps)
            frame_count = 0
            prev_frame = None

            # Extract video filename without extension
            video_filename = os.path.basename(video_path)
            video_name_no_ext = os.path.splitext(video_filename)[0]
            frame_save_count = 0

            for packet in container.demux(stream):
                for frame in packet.decode():
                    if frame_count % frame_interval_frames == 0:
                        frame_bgr = frame.to_ndarray(format='bgr24')

                        # 检查帧是否为乱码帧
                        frame_gray_full = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2GRAY)
                        std_dev = np.std(frame_gray_full)
                        min_val, max_val = frame_gray_full.min(), frame_gray_full.max()
                        pixel_range = max_val - min_val

                        if (std_dev < std_threshold and pixel_range < range_threshold):
                            continue  # 跳过乱码帧

                        if prev_frame is not None:
                            prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
                            frame_gray = frame_gray_full
                            difference = np.mean(np.abs(prev_frame_gray.astype("float") - frame_gray.astype("float")))

                            if difference > difference_threshold:
                                if filename_mode == "uuid":
                                    # 保存帧
                                    _name = f"{uuid.uuid4()}.jpg"
                                else:
                                    # filename_mode = "file"
                                    _name = f"{video_name_no_ext}_{frame_save_count}.jpg"

                                output_path = os.path.join(output_dir, _name)
                                cv2.imwrite(output_path, frame_bgr)
                                frame_save_count += 1
                                prev_frame = frame_bgr
                        else:
                            # 保存第一帧
                            if filename_mode == "uuid":
                                _name = f"{uuid.uuid4()}.jpg"
                            else:
                                # filename_mode = "file"
                                _name = f"{video_name_no_ext}_{frame_save_count}.jpg"

                            output_path = os.path.join(output_dir, _name)
                            cv2.imwrite(output_path, frame_bgr)
                            frame_save_count += 1
                            prev_frame = frame_bgr

                    frame_count += 1

            container.close()

        except Exception as e:
            tqdm.write(f"无法处理视频 {video_path}：{e}")
            continue


In [2]:

# # 243  视频有点问题
# video_dir = r"D:\ddesktop\monitoring\正泰电气\98铁芯仓库10_20241009111753"
# output_dir = r"D:\ddesktop\monitoring\frame_new"
# get_frames_with_pyav(video_dir, output_dir)

video_root = r"E:\monitoring_video"
output_dir = r"D:\ddesktop\monitoring\datadata\frames_2"
get_frames_with_pyav(video_root, output_dir)

Processing videos:   0%|          | 0/8 [00:00<?, ?video/s]


正在处理视频文件 (1/8): E:\monitoring_video\D02_20241024160802.mp4


Processing videos:  12%|█▎        | 1/8 [03:53<27:14, 233.49s/video]


正在处理视频文件 (2/8): E:\monitoring_video\D02_20241024161255.mp4


Processing videos:  25%|██▌       | 2/8 [35:32<2:01:18, 1213.16s/video]

...

Processing videos:  75%|███████▌  | 6/8 [1:13:20<22:10, 665.02s/video]


正在处理视频文件 (7/8): E:\monitoring_video\D04_20241024160752.mp4


Processing videos:  88%|████████▊ | 7/8 [1:27:27<12:04, 724.53s/video]


正在处理视频文件 (8/8): E:\monitoring_video\D04_20241024165733.mp4


Processing videos: 100%|██████████| 8/8 [1:39:55<00:00, 749.46s/video]
